In [ ]:
! pwd

In [ ]:
"""
python=3.7
did pip
tensorflow==2.3.1
nltk==3.5
colorama==0.4.3
numpy==1.18.5
scikit_learn==0.23.2
Flask==1.1.2

"""

In [ ]:
# Import

import json 
import numpy as np 

import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Open json file
with open('intents.json') as file:
    data = json.load(file)

# Var define
training_sentences = []
training_labels = []
labels = []
responses = []

# loop to load data 
for intent in data['intents']:
    for pattern in intent['patterns']:
        #
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    #
    responses.append(intent['responses'])

    # creates labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


num_classes = len(labels)

In [ ]:

# Transform Target Labels 

# define model 
lbl_encoder = LabelEncoder()
# fit model 
lbl_encoder.fit(training_labels)
# transform model 
training_labels = lbl_encoder.transform(training_labels)



 We can also add “oov_token” which is a value for “out of token” to deal with out of vocabulary words(tokens) at inference time.

In [ ]:
# 

# Var define
vocab_size = 1000
embedding_dim = 16
max_len = 20 
oov_token = '<00V>'

# declare tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
# fit tokenier
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequencees(training_sentences)
# creates equal sizes between the text sequences
padded_sequences = pad_sequences(sequences, truncating = 'post', maxlen = max_len)


In [ ]:
# Define NN Model

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, actication='relu'))
model.add(Dense(num_classes, activation = 'softmax'))


In [ ]:
# Compile model

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [ ]:
# Model Summory 

model.summary()

In [ ]:
# Train Model 

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels),  epochs=epochs)

# Making a Pickle
<hr>

In [ ]:
# save the trained model 
model.save('chat_model')

# save the fitted Tokenizer
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fittend label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


    